In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import json

%matplotlib inline

In [2]:
def softmax(x, temp=1):
#     x should be as long as number of possible clicks, each possibility given a value (
#     the x.max() is there for stability
    ex = np.exp((x - x.max()) / temp)
    return ex / ex.sum()

In [3]:
cd ..\experiments\data\human\1.8

C:\Users\piichan\Documents\Berkeley\CoCoSci Lab\mcrl\mcrl\experiments\data\human\1.8


In [4]:
df=pd.read_csv('trials.csv', sep=',')

In [5]:
import ast
toList = lambda x: ast.literal_eval(x)
df['clicks'] = df['clicks'].apply(toList)

In [50]:
cd ..\..\..\exp1\static\json

[WinError 3] The system cannot find the path specified: '../../../exp1/static/json'
C:\Users\piichan\Documents\Berkeley\CoCoSci Lab\mcrl\mcrl\experiments


In [41]:
with open('rewards_0.10.json') as f:
    rewards_low = json.load(f)

with open('rewards_1.25.json') as f:
    rewards_med = json.load(f)
    
with open('rewards_4.00.json') as f:
    rewards_high = json.load(f)

In [40]:
def get_rewards_for_trial(all_rewards, trial_id):
    for trial in all_rewards['train']:
        if trial['trial_id'] == trial_id:
            return trial['stateRewards']
    for trial in all_rewards['test']:
        if trial['trial_id'] == trial_id:
            return trial['stateRewards']

In [82]:
df.values[11]

array([11, 0, 1.25, 'none', 'none', 12, 6426.0, '[]', 15.0, 4,
       list([1, 5, 9, 13]), '[1272, 2030, 2884, 3586]', '[1, 13, 14, 15]',
       '[4572, 6033, 9770]'], dtype=object)

## Brute Force Approach

In [35]:
df.sort_values('info_cost')
low_cost = df.loc[df.info_cost == 0.1]
med_cost = df.loc[df.info_cost == 1.25]
high_cost = df.loc[df.info_cost == 4]

In [19]:
cost_type = low_cost
brfs_count = 0
dfs_count = 0
top = set([9, 10, 11, 12])
left = set([13, 14, 15, 16])
right = set([5,6,7,8])
bottom = set([1, 2, 3, 4])
for seq in cost_type['clicks']:
    if seq == []:
        continue     
    if len(seq) < 4:
        continue
    first_four = seq[:4]
    if set(first_four) == set([1,5,9,13]):
        if len(seq) > 4:
            rest = seq[4:]
            if len(rest) > 4:
                rest = seq[4:8]
            if set(rest) <= set([2,6,10,14]):
                brfs_count += 1 
        else:
            brfs_count += 1
    elif (set(first_four) == top)or(set(first_four) == right)or(set(first_four) == bottom)or(set(first_four) == left):
        if len(seq) > 4:
            rest = set(seq[4:])
            more = False
            if len(rest) > 4:
                more = True
                rest = set(seq[4:8])
            if (rest<=top)or(rest<=right)or(rest<=left)or(rest<=bottom):
                if more:
                    rest = set(seq[8:])
                    if len(rest) > 4:
                        rest = set(seq[8:12])
                    if (rest<=top)or(rest<=right)or(rest<=left)or(rest<=bottom):
                            dfs_count += 1                    
                else:
                    dfs_count += 1
        else:
            dfs_count += 1
    else: #between 1 and 3 clicks were made
        if (set(seq) < top)or(set(seq) < right)or(set(seq)< bottom) or (set(seq) < left):
            dfs_count += 1
        elif set(seq) < set([1, 5, 9, 13]):
            brfs_count += 1
        
(brfs_count/cost_type['clicks'].size) * 100

3.75

ver 0.991 

values are given in percentages; top to bottom is low, med, and high cost

i stands for initial (only the first 4 clicks) while f stands for full (entire sequence)

| Breadth-First | Depth-First(i)| Depth-First(f)|
| ------------- |:-------------:| -------------:|
| 3.02          | 12.50         |8.47       |
| 6.07          | 12.13         |    9.375      |
| 4.23          | 3.43          |    2.42       |



## Likelihood Model

In [59]:
def prune(threshold, state):
    #returns nodes on pruned branches
    pruned = []
    observed_states = np.nonzero(state)[0]
    bottom = [1, 2, 3, 4]
    top = [9, 10, 11, 12]
    right = [5, 6, 7, 8]
    left = [13, 14, 15, 16]
    direction = False
    if len(observed_states) != 0:
        for loc in observed_states:
            if (loc in [1, 2]) and (loc not in pruned):
                direction = bottom
            elif (loc in [5, 6]) and (loc not in pruned):
                direction = right
            elif (loc in [9, 10]) and (loc not in pruned):
                direction = top
            elif (loc in [13, 14]) and (loc not in pruned):
                direction = left
            if direction:
                path_val = np.sum(state[direction])
                if path_val < threshold:
                    pruned += direction
            else: #prune arm when both leaf nodes have a value below the threshold
                if (loc in [3, 7, 11, 15]) and (loc not in pruned):
                    if (state[loc] < threshold) and (state[loc+1] < threshold):
                        pruned += [loc-2, loc-1, loc, loc+1]
                elif (loc not in pruned):
                    if (state[loc] < threshold) and (state[loc-1] < threshold):
                        pruned += [loc-3, loc-2, loc-1, loc]
    return pruned        

In [60]:
#Note: Only continues depth-first based off of the last click 
#TODO: Check if half-explored arms are eventually explored to completion
def depth_first(state, cur_clicks, dist, strategy_t, prune_t):
    prob_next_click = np.zeros(17)
    observed_states = np.nonzero(state)[0] #works only because indices represent location
    possible_clicks = []
    if len(cur_clicks) == 0:
        possible_clicks += [1, 5, 9, 13]
    else:
        last_click = cur_clicks[-1]
        pruned_nodes = prune(prune_t, state)
        if last_click in [1, 5, 9, 13]:
            if last_click+1 in observed_states:
                for click in [last_click+2, last_click+3]:
                    if (click not in observed_states) and (click not in pruned_nodes):
                        possible_clicks.append(click)
                if len(possible_clicks) == 0:
                    for click in [1, 5, 9, 13]:
                        if (click not in observed_states) and (click not in pruned_nodes):
                            possible_clicks.append(click)
            else:
                if last_click+1 not in pruned_nodes:
                    possible_clicks.append(last_click+1)
                else:
                    click_candidates = [1, 5, 9, 13]
                    click_candidates.remove(last_click)
                    for c in click_candidates:
                        if (c in observed_states) or (c in pruned_nodes):
                            click_candidates.remove(c)
                    possible_clicks += click_candidates
        else:
            if last_click+1 in observed_states:
                if last_click+2 in observed_states:
                    for click in [1, 5, 9, 13]:
                        if (click not in observed_states) and (click not in pruned_nodes):
                            possible_clicks.append(click)
                elif (last_click+2 < 17) and (last_click+2 not in pruned_nodes):
                    possible_clicks.append(last_click+2)
                else:
                    for click in [1, 5, 9, 13]:
                        if (click not in observed_states) and (click not in pruned_nodes):
                            possible_clicks.append(click)
            elif (last_click+1 < 17) and (last_click+1 not in pruned_nodes):
                possible_clicks.append(last_click+1)
            else:
                for click in [1, 5, 9, 13]:
                        if (click not in observed_states) and (click not in pruned_nodes):
                            possible_clicks.append(click)
    if len(possible_clicks) != 0:
        for possibility in possible_clicks:
            if dist == 'Uniform':
                prob_next_click[possibility] = 1/len(possible_clicks)
            elif dist == 'Softmax':
                prob_next_click[possibility] = 1
    return prob_next_click

In [61]:
test_state = np.array([0,1,5,0,20,0,0,0,0,0,0,0,0,1,0,-10,-10])
res = depth_first(test_state, [], 'Uniform', None, -4)
res

array([ 0.  ,  0.25,  0.  ,  0.  ,  0.  ,  0.25,  0.  ,  0.  ,  0.  ,
        0.25,  0.  ,  0.  ,  0.  ,  0.25,  0.  ,  0.  ,  0.  ])

In [62]:
def breadth_first(state, cur_clicks, dist, strategy_t, prune_t):
    prob_next_click = np.zeros(17)
    observed_states = np.nonzero(state)[0]
    possible_clicks = []
    if len(cur_clicks) == 0:
        possible_clicks += [1, 5, 9, 13]
    else:
        last_click = cur_clicks[-1]
        pruned_nodes = prune(prune_t, state)
        if last_click in [1, 5, 9, 13]:
            if set([1, 5, 9, 13]).issubset(set(observed_states)):
                if set([2, 6, 10, 14]).issubset(set(observed_states)):
                    for click in [3, 4, 7, 8, 11, 12, 15, 16]:
                        if (click not in observed_states) and (click not in pruned_nodes):
                            possible_clicks.append(click)
                else:
                    for click in [2, 6, 10, 14]:
                        if (click not in observed_states) and (click not in pruned_nodes):
                                possible_clicks.append(click)
            else:
                for click in [1, 5, 9, 13]:
                    if (click not in observed_states) and (click not in pruned_nodes):
                            possible_clicks.append(click)
        elif last_click in [2, 6, 10, 14]:
            if set([2, 6, 10, 14]).issubset(set(observed_states)):
                for click in [3, 4, 7, 8, 11, 12, 15, 16]:
                    if (click not in observed_states) and (click not in pruned_nodes):
                        possible_clicks.append(click)
            else:
                for click in [2, 6, 10, 14]:
                    if (click not in observed_states) and (click not in pruned_nodes):
                        possible_clicks.append(click)
        else:
            for click in [3, 4, 7, 8, 11, 12, 15, 16]:
                    if (click not in observed_states) and (click not in pruned_nodes):
                        possible_clicks.append(click)
    if len(possible_clicks) != 0:
        for possibility in possible_clicks:
            if dist == 'Uniform':
                prob_next_click[possibility] = 1/len(possible_clicks)
            elif dist == 'Softmax':
                prob_next_click[possibility] = 1
    return prob_next_click

In [63]:
test_state = np.array([0,1,5,0,20,5,0,0,0,-3,0,0,0,1,0,-10,-10])
res = breadth_first(test_state, [1, 4, 16, 2, 13], 'Uniform', None, 0)
res

array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.])

In [64]:
#assumes that average reward value is 0
def best_first(state, cur_clicks, dist, strategy_t, prune_t):
    prob_next_click = np.zeros(17)
    observed_states = np.nonzero(state)[0]
    possible_clicks = []
    pruned_nodes = prune(prune_t, state)
    expected_reward_value = 0 #or self.env.reward.expectation()
    if len(cur_clicks) == 0:
        possible_clicks += [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
    else:
        for i in np.arange(1,17):
            if (i not in observed_states) and (i not in pruned_nodes):
                possible_clicks.append(i)
    actual_state = np.copy(state)
    for i in np.arange(1, 17):
        if state[i] == 0:
            state[i] = expected_reward_value
    if len(possible_clicks) != 0:
        for possibility in possible_clicks:
            if possibility in [1, 5, 9, 13]:
                indices = [possibility+1, possibility+2]
                path_1 = np.sum(state[indices])
                indices[1] += 1
                path_2 = np.sum(state[indices])
                prob_next_click[possibility] = np.maximum(path_1, path_2)
            elif possibility in [2, 6, 10, 14]:
                indices = [possibility-1, possibility+1]
                path_1 = np.sum(state[indices])
                indices[1] += 1
                path_2 = np.sum(state[indices])
                prob_next_click[possibility] = np.maximum(path_1, path_2)
            elif possibility in [4, 8, 12, 16]:
                indices = [possibility-1, possibility-3]
                prob_next_click[possibility] = np.sum(state[indices])
            else:
                indices = [possibility-1, possibility-2]
                prob_next_click[possibility] = np.sum(state[indices])
#         sum_of_paths = np.sum([np.exp(n) for n in prob_next_click])
#         for j in np.arange(len(prob_next_click)):
#             path_val = prob_next_click[j]
#             prob_next_click[j] = np.exp(path_val)/sum_of_paths #softmax function
#             if j in observed_states:
#                 prob_next_click[j] = 0
    return prob_next_click

In [65]:
#state = np.array([0, 0, 20, 0, 0, 12, -8, 0, 0, -4, 0, 0, 0, 10, 0, 0, 0])
test_state = np.array([0,1,5,0,20,5,0,0,0,-3,0,0,0,1,0,-10,-10])
res = best_first(test_state, [1, 4, 16, 2, 13], 'Uniform', None, -30)
res

array([ 0.,  0.,  0.,  6.,  0.,  0.,  5.,  5.,  5.,  0., -3., -3., -3.,
        0., -9.,  0.,  0.])

In [66]:
#needs to be tested
def backwards_best(state, cur_clicks, dist, strategy_t, prune_t):
    prob_next_click = np.zeros(17)
    observed_states = np.nonzero(state)[0]
    possible_clicks = []
    pruned_nodes = prune(prune_t, state)
    leaves = [3, 4, 7, 8, 11, 12, 15, 16]
    goal = False
    goal_node = None
    if len(cur_clicks) == 0:
        possible_clicks += leaves
    else:
        for leaf in leaves: # should the leaves be examined in the order clicked?
            if (leaf-2 not in pruned_nodes) and (state[leaf] > strategy_t): #a goal node has been identified
                #check that the whole arm hasn't been explored before marking goal as true
                if leaf in [3, 7, 11, 15]:
                    if (leaf-1 not in observed_states) or (leaf-2 not in observed_states):
                        goal = True
                        goal_node = leaf
                        break
                else:
                    if (leaf-2 not in observed_states) or (leaf-3 not in observed_states):
                        goal = True
                        goal_node = leaf
                        break
        if goal:
            #go backwards on that arm
            if goal_node in [3, 7, 11, 15]:
                if goal_node-1 not in observed_states:
                    possible_clicks.append(goal_node-1)
                else:
                    possible_clicks.append(goal_node-2)
            else:
                if goal_node-2 not in observed_states:
                    possible_clicks.append(goal_node-2)
                else:
                    possible_clicks.append(goal_node-3)
        else:
            for leaf in leaves:
                if (leaf-2 not in pruned_nodes) and (leaf not in observed_states):
                    possible_clicks.append(leaf)
            
    if len(possible_clicks) != 0:
        for possibility in possible_clicks:
            if possibility in [1, 5, 9, 13]:
                indices = [possibility+1, possibility+2]
                path_1 = np.sum(state[indices])
                indices[1] += 1
                path_2 = np.sum(state[indices])
                prob_next_click[possibility] = np.maximum(path_1, path_2)
            elif possibility in [2, 6, 10, 14]:
                indices = [possibility-1, possibility+1]
                path_1 = np.sum(state[indices])
                indices[1] += 1
                path_2 = np.sum(state[indices])
                prob_next_click[possibility] = np.maximum(path_1, path_2)
            elif possibility in [4, 8, 12, 16]:
                indices = [possibility-1, possibility-3]
                prob_next_click[possibility] = np.sum(state[indices])
            else:
                indices = [possibility-1, possibility-2]
                prob_next_click[possibility] = np.sum(state[indices])
    return prob_next_click

In [67]:
def satisficing_stop(state, threshold):
    bottom = [1, 2, 3, 4]
    top = [9, 10, 11, 12]
    right = [5, 6, 7, 8]
    left = [13, 14, 15, 16]
    for direction in [bottom, top, right, left]:
        observed_value = np.sum(state[direction])
        if observed_value > threshold:
            return True
    return False    

In [101]:
def strategy_likelihood(click_seq, strategy, cost_per_click, rewards, p_error, soft_temp=1,
                        log=False, dist='Softmax', strategy_t=None, prune_t=-30, stop=(0,0)):
    #Stop should be a tuple of (stopping rule type, threshold)
    if log:
        prob = 0
    else:
        prob = 1
    cur_state = np.zeros(17)
    cur_clicks = []
    stop_type = stop[0]
    stop_threshold = stop[1]
    for click in click_seq:
        possible_next_clicks = strategy(cur_state, cur_clicks, dist, strategy_t, prune_t)
        next_state = np.copy(cur_state)
        next_state[click] = rewards[click]
        cur_clicks.append(click)
        n_possible_clicks = np.count_nonzero(possible_next_clicks)
        if stop_type == 1: # number of clicks x cost per click
            if (len(cur_clicks)*cost_per_click) > stop_threshold:
                possible_next_clicks = np.zeros(17)
        elif stop_type == 2: #satisficing
            stop_flag = satisficing_stop(cur_state, stop_threshold)
            if stop_flag:
                possible_next_clicks = np.zeros(17)
        if dist == 'Uniform':
            p_strat = possible_next_clicks[click] 
            num_wrong_clicks = 16 - (np.count_nonzero(cur_state)+n_possible_clicks)
            p_click_given_state = (((1-p_error)*p_strat) + (p_error/num_wrong_clicks))
        elif dist == 'Softmax':
            prob_next_clicks = softmax(possible_next_clicks, soft_temp)
            p_click_given_state = prob_next_clicks[click]
        if log:
            prob += np.log(p_click_given_state)
        else:
            prob *= p_click_given_state
        cur_state = next_state
    return prob

In [102]:
strategy_likelihood([1, 2, 3, 4], breadth_first, 0.01, [0, 1, 2, 3, 4, 5], 0.1, log=True)

-10.737125851279629

In [91]:
participant = df.values[11]
rewards = get_rewards_for_trial(rewards_med, participant[6])
strategy_likelihood(participant[10], breadth_first, participant[2], rewards, 0.1, log=True)

-8.2176359933709033

In [30]:
def strategy_features(state, last_click, action, expansion_rule):
    #returns an array of indicator values for each feature; value is 1 if action is compatible with the strategy
    ''' Features:
        [0] is_term
        [1] term_reward
        [2] voi_myopic
        [3] vpi_action
        [4] vpi_full
        [5] quality_ev
        [6] quality_std
        [7] depth
        [8] same_branch_as_last
    '''
    features = np.zeros(9)
    prob_next_click = expansion_rule(state, [last_click], 'Uniform', None, -30)
    func_name = expansion_rule.__name__
    if func_name == 'best_first':
        max_prob = np.maximum(prob_next_click)
        if prob_next_click[action] == max_prob:
            features[5] = 1
    elif func_name == 'breadth_first':
        if prob_next_click[action] > 0:
            features[7] = -1
    elif func_name == 'depth_first':
        if prob_next_click[action] > 0:
            features[6] = -1
            features[7] = -1
    
    return features

In [31]:
strategy_features(np.array([0, 1, 2, 0, 0, 0, 0,0,0,0,0,0,0,0,0,0,0]), 2, 3, depth_first)

array([ 0.,  0.,  0.,  0.,  0.,  0., -1., -1.,  0.])